In [1]:
import tensorflow as tf
print(f'tensorflow v{tf.__version__}')

tensorflow v2.5.0-dev20210213


In [2]:
from keras.preprocessing import sequence
import keras
import os
import numpy as np

### Loading Shakespare Dataset

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [5]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### Encoding

Since encoding each word will be harder for us, we will encode each unique character as a different integer instead

In [6]:
# sort each unique character in the text
vocab = sorted(set(text)) 

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def encode(text):
  return np.array([char2idx[c] for c in text])

def decode(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

In [7]:
dataset = encode(text)

In [8]:
print("Text:", text[:13])
print("Encoded:", encode(text[:13]))
print("Decoded:", decode(dataset[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]
Decoded: First Citizen


### Training

In [9]:
seq_length = 100  
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(dataset)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [10]:
def split_input_target(chunk):  
    input_text = chunk[:-1]  
    target_text = chunk[1:] 
    return input_text, target_text 

dataset = sequences.map(split_input_target)

In [11]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

### Building the model

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]), # None - dont know how long each one will be
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True, # if False, will only return a single output
                        stateful=True,
                        recurrent_initializer='glorot_uniform'), # what the values start at
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


### Loss/Cost Function

In [13]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch) 
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)") 

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [14]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions.shape)
pred = example_batch_predictions[0]

64
(64, 100, 65)


In [15]:
# Output the sample distribution
sampled_indices = tf.random.categorical(pred, num_samples=1)

#Reshape the array and convert all the int to number to see the actual character
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
# Decode the array from int to text
predicted_chars = decode(sampled_indices)

predicted_chars 

"MjmuQS,JSFVcGRp!s?JXEE\n\nTKi$UZZqVo3RTID..kGMOftTSnlWoQt'MDPzYeXO3s&;sCQMXRoUzmkQkup:&:L?,3!VwfSmzfEh"

In [16]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [17]:
model.compile(optimizer='adam', loss=loss)

In [18]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

early_stop = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=4, restore_best_weights=True)

### Training

Train as many epochs as you like, it's not possible to overfit this model

In [19]:
history = model.fit(data, epochs=2, callbacks=[checkpoint_callback, early_stop])
# history = model.fit(data, epochs=10, callbacks=[checkpoint_callback, early_stop])
# history = model.fit(data, epochs=50, callbacks=[checkpoint_callback, early_stop])

Epoch 1/2
172/172 [==============================] - 26s 142ms/step - loss: 2.9674
Epoch 2/2
172/172 [==============================] - 25s 142ms/step - loss: 1.9269


### Loading the Model

In [20]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [21]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [22]:
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/50_epoch"))˜

In [23]:
# Loading a specific checkpoint
# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

### Generating Text

In [24]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [25]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: Romeo 
Romeo pright cail thy tconco's.

And,
Such sor. Mary!

LORCOO:
To love
Why come, sin a brok lide incuerhrets: bean?

LARTIO:
In tian? I flat lere o'cl plasing's Clerise: I dederth,
And I must I umone with yours in must Bolk,
At am growedt, sool more; for the Proforr?
'tim, his mist the honstato as diend.

SSBANINAN:
I to you peepunr:
You then os sasensly caling this innels.
Where tend thee; My ware, I'll paspears what I dove wonds well,
Think wore! Do tod chail dismestimarn,
No i atay to be. I mur made me; if Clayem ene
Of thould sweateral maringly.

Glond I peape up to sue 'dy talk!

MARCEASI:
The lighes wave I propar Goke, not, fill his langel-was.

WAAND CLARUS:
To train, myself,
And I cone the our brest; hore insiclicial enclais
With brighieg, I wous, no the will flanks!
I most revell
Wo wer
